In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels
import scipy
import os
import datetime


In [ ]:
# Read in the vehicle GPS data (input from user)
filepath = 'taxi_gps_data/data/'
files = glob.glob(os.path.join(filepath, '*.csv'))
all_vehicles = {}
for file in files:
    #take extension out of filename
    filename = file.split('.')[0]
    #take filepath out of filename
    filename = filename.split('/')[-1]
    #save the data to a dictionary
    all_vehicles[filename] = pd.read_csv(file)